<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from numba import njit, int64, float64, types, typeof
from numba.types import Array, string, boolean, UniTuple, Tuple, DictType
from numba.experimental import jitclass

np.random.seed(42)
x = np.random.randn(1000000)
y = np.random.randn(1000000)
z = np.random.randn(1000000)

# Example of "human-readable" signals
entry_sig_ = ((x, y, "crossup", False),)
exit_sig_ = ((x, z, "crossup", False), "or_", (x, y, "crossdown", False))

# Turn signals into homogeneous tuple
# entry_sig_
entry_sig = (((x, y, "crossup", False), "NOP"),)
# exit_sig_
exit_sig = (((x, z, "crossup", False), "or_"), ((x, y, "crossdown", False), "NOP"))


@njit
def crossup(x, y, i):
    """
    x,y: np.array
    i: int - point in time
    Returns: 1 or 0 when condition is met
    """
    if x[i - 1] < y[i - 1] and x[i] > y[i]:
        out = True
    else:
        out = False
    return out


@njit
def crossdown(x, y, i):
    if x[i - 1] > y[i - 1] and x[i] < y[i]:
        out = True
    else:
        out = False
    return out

sig_dt = Tuple([Tuple([Array(float64, 1,"C"), Array(float64, 1, "C"), string, boolean]), string])


spec = [("memory", boolean[:]), ("signals", DictType(int64, sig_dt)), ("L", int64)]


@jitclass(spec)
class MultiSig:
    def __init__(self, signals):
        L = len(signals)
        self.memory = np.array([False] * L)
        self.signals = {0: signals[0]}
        for i in range(1, L):
            self.signals[i] = signals[i]
        self.L = L

    def single_sig(self, signal, i, n):
        """
        Accepts:
            - signal: tuple of 4 fields
            - i: int - point in time
            - n: int - consequetive number of the signal
        Updates:
            - corresponding memory field
        Returns:
            - boolean accounting for memory
        """
        x, y, how, acc = signal
        if how == "crossup":
            out = crossup(x, y, i)
        elif how == "crossdown":
            out = crossdown(x, y, i)
        out = out | self.memory[n]
        if acc:
            self.memory[n] = out
        return out

    def sig(self, i):
        s, logic = self.signals[0]
        out = self.single_sig(s, i, 0)
        for cnt in range(1, self.L):
            s = self.single_sig(self.signals[cnt][0], i, cnt)
            out = out | s if logic == "or_" else out & s
            logic = self.signals[cnt][1]
        return out

    def reset(self):
        self.memory = np.array([False] * self.L)

In [2]:
ms = MultiSig(entry_sig)
ms.sig(11)

False